In [15]:
import pandas as pd
import tensorflow as tf

In [16]:
ontology_and_seq__fp = "../data/intermediary/zebrafish_protein_ontology_and_seqs.tsv"
df = pd.read_csv(ontology_and_seq__fp, index_col=0)[["seq", "go_name"]]
df.go_name = pd.Categorical(df.go_name)

In [17]:
dataset = tf.data.Dataset.from_tensor_slices((df.seq, df.go_name))

In [18]:
df.dtypes

seq          object
go_name    category
dtype: object